<h2> Spectroscopic data reduction : spectra module

<h2> ARCES : echelle spectroscopy

In [1]:
from pyvista import imred, tv, spectra
import pyvista.data
from importlib_resources import files
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

pyautogui does not seem to be available, disabling arrow key cursor moves


pyvista uses a display tool defined in the tv module. To use the interactive
display in a notebook, set the display to be an external display window, e.g. with 
<code>
%matplotlib qt
</code>
Instantiate a tv object, here we just call it t, but you could call it whatever you want!

In [2]:
%matplotlib qt
t=tv.TV()

The basic tool for basic image reduction is a Reducer object, defined in the imred module. Instantiate a reducer here. The main argument is an instrument name, which tells it to read a YAML configuration file for the specified instrument. We also give it an optional dir= argument to specify the default directory from which to read images, if a directory is not specified in subsequent commands that read images.

In [159]:
red=imred.Reducer('ARCES',dir='UT211031/echelle')

INSTRUMENT: ARCES   config: 
  will use format:  UT211031/echelle/*{:04d}.f*.fits*
         gain:  [3.8]    rn: [ 7.]
         scale:  None   
  Biastype : 0
  Bias box: 
    SC    NC    SR    NR
  2075    51    20  2009 
  Trim box: 
    SC    NC    SR    NR
    22  2026     0  2048 
  Norm box: 
    SC    NC    SR    NR
  1000    51  1000    51 


A main method of the reducer object is the reduce() method. Without any additional arguments, reduce() will read an image from disk, subtract the overscan (region(s) as determined from the instrument configuration file), compute an uncertainty array using the gain and readout noise from the instrument configuration file, and return a CCDData object with the data, uncertainty, and mask. 
<p>
To specify the input image, we could pass a string with the file name. If the string does not include a '/', it will read from the default input directory.
<p>
If the file can be identified with a unique integer, then you can just specify this number, which can be very convenient. This is turned into a character string using the formstr attribute define in the configuration file, which is used to search for the file to read.
<p>
We can display the image using the tv() method of our display tool, which can take as input a Data object, and numpy array, or a FITS HDU object.

In [160]:
a=red.reduce(28)

t.tv(a)

  Reading file: UT211031/echelle/HIP109807.0028.fits
  subtracting overscan:  1291.72
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


<h4> Calibration: make and apply flat field

If we add additional arguments to reduce(), we can add additional calibration steps. For example, to flat field the data, we would add a flat= keyword through which we give the reducer a flat field. To add a spatial bias subtraction, we would add a bias= keyword through which we give the reducer a superbias frame.
<br>
First, however, we have to make the calibration products, which is accomplished using the mkflat(), mkbias(), etc methods. These take as input a list of frames to be used to construct the master calibration frame (e.g.. superflat). For ARCES, we will keep the spectral shape of the flats for now.

Create biases and flats. Note that for flats, we have to do scattered light removal, which can be done on reduction of individual images, but since it is slow, we will do it on the combined flat. If we add the display= keyword, giving a display object, then the calibration frames will be displayed, showing each input frame relative to the master frame, so you can inspect and make sure that bad frames are not being included in the combination.

In [161]:
bias=red.mkbias([19,20,21,22,23,24])
flat_red=red.mkflat([5,6,7,8,9,10,11])
flat_blue=red.mkflat([12,13,14,15,16,17,18])

# do scatter after combination to save time
red.scatter(flat_red,scat=red.scat,display=t)
red.scatter(flat_blue,scat=red.scat,display=t)

  Reading file: UT211031/echelle/bias.0019.fits
  subtracting overscan:  1288.79
  Reading file: UT211031/echelle/bias.0020.fits
  subtracting overscan:  1288.58
  Reading file: UT211031/echelle/bias.0021.fits
  subtracting overscan:  1288.52
  Reading file: UT211031/echelle/bias.0022.fits
  subtracting overscan:  1288.55
  Reading file: UT211031/echelle/bias.0023.fits
  subtracting overscan:  1288.97
  Reading file: UT211031/echelle/bias.0024.fits
  subtracting overscan:  1289.03
  combining data with median....
  calculating uncertainty....
  Reading file: UT211031/echelle/flat_red.0005.fits
  subtracting overscan:  1287.67
  Reading file: UT211031/echelle/flat_red.0006.fits
  subtracting overscan:  1287.76
  Reading file: UT211031/echelle/flat_red.0007.fits
  subtracting overscan:  1287.53
  Reading file: UT211031/echelle/flat_red.0008.fits
  subtracting overscan:  1287.68
  Reading file: UT211031/echelle/flat_red.0009.fits
  subtracting overscan:  1287.39
  Reading file: UT211031/e

Combine the red and blue flats.

In [162]:
flat=flat_red.multiply(flat_red.header['MEANNORM'])
flat=flat.add(flat_blue.multiply(flat_blue.header['MEANNORM']))
flat=flat.divide(flat_red.header['MEANNORM']+flat_blue.header['MEANNORM'])
t.clear()
t.tv(flat_blue)
t.tv(flat_red)
t.tv(flat)
t.tv(flat,sn=True)

Read and display a star spectral image. For ARCES, we do not apply a flat field here, since we only have flats with the orders, which can move around a bit. Instead, we will use "1D" flats later in the processing.

In [163]:
star=red.reduce(28,bias=bias)
t.tv(star,max=1000) 

  Reading file: UT211031/echelle/HIP109807.0028.fits
  subtracting overscan:  1291.72
  subtracting bias...
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


In [164]:
crstar=red.crrej(star,crbox='lacosmic',display=t,objlim=50)

  zapping CRs with ccdproc.cosmicray_lacosmic
  See CRs and CR-zapped image and original using - key
   To continue, hit space in display window (p for debug) 


<h4> Tracing and extraction

In [165]:
traces=spectra.Trace('ARCES/ARCES_traces.fits')
vars(traces)

no attribute:  rows


{'type': 'Polynomial1D',
 'degree': 2,
 'sc0': 1064,
 'pix0': -0.016310744310946035,
 'spectrum': array([  7.71968625e-04,   7.23036355e-04,   8.62288638e-04, ...,
          7.60172697e-05,   3.69138026e-04,   1.36280447e-04], dtype=float32),
 'rad': 5,
 'lags': array([-10,  -9,  -8,  -7,  -6,  -5,  -4,  -3,  -2,  -1,   0,   1,   2,
          3,   4,   5,   6,   7,   8,   9]),
 'transpose': False,
 'rows': None,
 'index': array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 1

For tracing and wavelength calibration, we will use previously determined traces and line identification as a first guess.

In [166]:
traces.retrace(flat,display=t)

Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shift:  0.0450065159434
Using shif

/home/holtz/git/pyvista/python/pyvista/spectra.py:987: RuntimeWarning: invalid value encountered in true_divide
  if gaussian :
/home/holtz/git/pyvista/python/pyvista/spectra.py:998: RuntimeWarning: invalid value encountered in true_divide
  if gaussian:
/home/holtz/git/pyvista/python/pyvista/spectra.py:987: RuntimeWarning: invalid value encountered in true_divide
  if gaussian :
/home/holtz/git/pyvista/python/pyvista/spectra.py:998: RuntimeWarning: invalid value encountered in true_divide
  if gaussian:


/home/holtz/git/pyvista/python/pyvista/spectra.py:987: RuntimeWarning: invalid value encountered in true_divide
  if gaussian :
/home/holtz/git/pyvista/python/pyvista/spectra.py:998: RuntimeWarning: invalid value encountered in true_divide
  if gaussian:
/home/holtz/git/pyvista/python/pyvista/spectra.py:987: RuntimeWarning: invalid value encountered in true_divide
  if gaussian :
/home/holtz/git/pyvista/python/pyvista/spectra.py:998: RuntimeWarning: invalid value encountered in true_divide
  if gaussian:


/home/holtz/git/pyvista/python/pyvista/spectra.py:987: RuntimeWarning: invalid value encountered in true_divide
  if gaussian :
/home/holtz/git/pyvista/python/pyvista/spectra.py:998: RuntimeWarning: invalid value encountered in true_divide
  if gaussian:
/home/holtz/git/pyvista/python/pyvista/spectra.py:987: RuntimeWarning: invalid value encountered in true_divide
  if gaussian :
/home/holtz/git/pyvista/python/pyvista/spectra.py:998: RuntimeWarning: invalid value encountered in true_divide
  if gaussian:


/home/holtz/git/pyvista/python/pyvista/spectra.py:987: RuntimeWarning: invalid value encountered in true_divide
  if gaussian :
/home/holtz/git/pyvista/python/pyvista/spectra.py:998: RuntimeWarning: invalid value encountered in true_divide
  if gaussian:
/home/holtz/git/pyvista/python/pyvista/spectra.py:987: RuntimeWarning: invalid value encountered in true_divide
  if gaussian :
/home/holtz/git/pyvista/python/pyvista/spectra.py:998: RuntimeWarning: invalid value encountered in true_divide
  if gaussian:


...

/home/holtz/git/pyvista/python/pyvista/spectra.py:987: RuntimeWarning: invalid value encountered in true_divide
  if gaussian :
/home/holtz/git/pyvista/python/pyvista/spectra.py:998: RuntimeWarning: invalid value encountered in true_divide
  if gaussian:
/home/holtz/git/pyvista/python/pyvista/spectra.py:987: RuntimeWarning: invalid value encountered in true_divide
  if gaussian :
/home/holtz/git/pyvista/python/pyvista/spectra.py:998: RuntimeWarning: invalid value encountered in true_divide
  if gaussian:


/home/holtz/git/pyvista/python/pyvista/spectra.py:987: RuntimeWarning: invalid value encountered in true_divide
  if gaussian :
/home/holtz/git/pyvista/python/pyvista/spectra.py:998: RuntimeWarning: invalid value encountered in true_divide
  if gaussian:
/home/holtz/git/pyvista/python/pyvista/spectra.py:987: RuntimeWarning: invalid value encountered in true_divide
  if gaussian :
/home/holtz/git/pyvista/python/pyvista/spectra.py:998: RuntimeWarning: invalid value encountered in true_divide
  if gaussian:


/home/holtz/git/pyvista/python/pyvista/spectra.py:987: RuntimeWarning: invalid value encountered in true_divide
  if gaussian :
/home/holtz/git/pyvista/python/pyvista/spectra.py:998: RuntimeWarning: invalid value encountered in true_divide
  if gaussian:
/home/holtz/git/pyvista/python/pyvista/spectra.py:987: RuntimeWarning: invalid value encountered in true_divide
  if gaussian :
/home/holtz/git/pyvista/python/pyvista/spectra.py:998: RuntimeWarning: invalid value encountered in true_divide
  if gaussian:


/home/holtz/git/pyvista/python/pyvista/spectra.py:987: RuntimeWarning: invalid value encountered in true_divide
  if gaussian :
/home/holtz/git/pyvista/python/pyvista/spectra.py:998: RuntimeWarning: invalid value encountered in true_divide
  if gaussian:



  See trace. Hit space bar to continue....


Sum up the arc lamp exposures, get the shift for the existing traces, and extract. Note that if you have a multiprocessor machine, you can specify number of threads to use for the extraction, which will speed things up (but the default threads=0 isn't too terrible).

In [167]:
arc=red.sum([1,2,3,4],crbox=[5,1])
traces.find(arc,display=t)
arcec=traces.extract(arc,display=t,threads=0)

  Reading file: UT211031/echelle/ThAr.0001.fits
  subtracting overscan:  1287.78
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  zapping CRs with filter [5,1]...
  Reading file: UT211031/echelle/ThAr.0002.fits
  subtracting overscan:  1287.78
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  zapping CRs with filter [5,1]...
  Reading file: UT211031/echelle/ThAr.0003.fits
  subtracting overscan:  1287.61
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  zapping CRs with filter [5,1]...
  Reading file: UT211031/echelle/ThAr.0004.fits
  subtracting overscan:  1287.77
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]
  zapping CRs with filter [5,1]...
  combining data with sum....
  calculating uncertainty....
  See spectra and cross-correlation. Hit any key in display window to continue..

Do line identification based on previously identified lines, and wavelength fit.

In [169]:
wav=spectra.WaveCal('ARCES/ARCES_wave.fits')
vars(wav)

  rms:    0.019
rejecting 90 points from 3653 total: 
  rms:    0.019
rejecting 90 points from 3653 total: 


{'type': 'chebyshev2D',
 'degree': 2,
 'ydegree': 3,
 'waves': array([ 10133.5662,   9833.4229,   9826.4492, ...,   3561.0304,
          3559.5081,   3545.5957]),
 'waves_order': array([ 56,  58,  58, ..., 160, 160, 160]),
 'orders': array([ 54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
         67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
         80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
         93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
        106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
        119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
        132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
        145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
        158, 159, 160]),
 'index': None,
 'pix0': -5.1791778332699323e-07,
 'pix': array([  979.6753688 ,   582.69743052,   639.72191686, ...,   678.21538205,
          712.43607968, 

In [170]:
wav.identify(spectrum=arcec,rad=3,display=t,plot=True)

  See spectrum and template spectrum (top), cross correlation(bottom) Figure(1200x850)


/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


  See identified lines.
  rms:    0.028
rejecting 25 points from 2012 total: 
  rms:    0.012
rejecting 26 points from 2012 total: 
  rms:    0.012
rejecting 26 points from 2012 total: 
  See 2D wavecal fit. Enter space in plot window to continue


KeyboardInterrupt: 

Now reduce an image

In [104]:
im=red.reduce(28,scat=red.scat,display=t)

  Reading file: UT211031/echelle/HIP109807.0028.fits
  subtracting overscan:  1291.72
  See bias box and cross section. 
   To continue, hit space in display window (p for debug) 
  estimating scattered light ...
    fitting surface ...
    rejecting points ...
    refitting surface ...
  See image with scattered light points
   To continue, hit space in display window (p for debug) 
  See scattered light image
   To continue, hit space in display window (p for debug) 


In [141]:
t.tv(im)

Get shift of traces, and extract. Alternatively, you could use a single call to retrace(), which will do the find() and then trace() using the shifted stored model as a starting guess.

In [142]:
traces.find(im,plot=t)
imec=traces.extract(im,plot=t)

  See spectra and cross-correlation. Hit any key in display window to continue....
  See extraction window(s). Hit space bar to continue....



In [143]:
t.clear()
t.tv(im)
t.tv(imec)

Get the wavelengths for all pixels from the wavelength solution and plot extracted spectra.

In [128]:
wav.add_wave(imec)
plt.figure()
for row in range(len(imec.wave)) :
    plt.plot(imec.wave[row],imec.data[row])

Now do 1D flat fielding. 2D flat fielding would be better, but poses challenges with ARCES because the traces move, and there's no way to illuminate all pixels.

In [130]:
traces.find(flat)
flat1d=traces.extract(flat)
t.tv(flat1d)

In [125]:
import copy
test=copy.deepcopy(imec)
test.data /= flat1d.data
test.uncertainty.array /= flat1d.data

In [140]:
t.tv(imec)
t.tv(test)

Resample onto logarithmic wavelength grid and combine orders

In [138]:
wnew=10**np.arange(3.5,4.0,5.5e-6)
comb=wav.scomb(imec,wnew,average=False,usemask=True)
plt.figure()
plt.plot(wnew,comb.data)
comb=wav.scomb(test,wnew,average=True,usemask=True)
plt.figure()
plt.plot(wnew,comb.data)


/home/holtz/git/pyvista/python/pyvista/spectra.py:650: RuntimeWarning: invalid value encountered in true_divide
  out = out / sig
/home/holtz/git/pyvista/python/pyvista/spectra.py:651: RuntimeWarning: divide by zero encountered in true_divide
  sig = np.sqrt(1./sig)


In [ ]:
from pyvista import reduce
fig=plt.figure()
reduce.all('oct21.yml',groups='ARCES',display=None,plot=fig)